In [ ]:
import configparser
import pymysql
from matplotlib.backends.backend_pdf import PdfPages
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
local_root = os.getcwd()

In [ ]:
# connect to db
config_path = os.path.join(local_root, "config.cfg")
cp = configparser.ConfigParser()
cp.read(config_path)
mysql_user = cp['MYSQL']['user']
mysql_password = cp['MYSQL']['password']
conn = pymysql.connect(user='root', password=mysql_password)
cursor = conn.cursor()


In [ ]:
# want to get to grouping legilators by who votes for whom - Done
# start, take single legislator, map who votes for their bills - Done
# start, take singe bill, see who votes for it - Done
# get all bills sponsered by single legislator,  - Done
# how often do two legislators vote the same way
# which legislators change votes on versions most

In [ ]:
# Get all of votes on bills sponsored by the legislator
def get_votes(sponsor, author_type='sponsor'):
    author_types = ['sponsor', 'lead', 'primary']
    if author_type not in author_types:
        raise ValueError("Valid author_types: {0}".format(", ".join(author_types)))
        
    sponsorship = ("select a.bill_id from capublic.bill_tbl a" + 
                   " join capublic.bill_version_authors_tbl b" + 
                   " where a.latest_bill_version_id = b.bill_version_id" +
                   " and  b.name = %s;") 
    
    lead_author = ("select a.bill_id from capublic.bill_tbl a" + 
                   " join capublic.bill_version_authors_tbl b" + 
                   " where a.latest_bill_version_id = b.bill_version_id" +
                   " and  b.name = %s" +
                   " and contribution = 'LEAD_AUTHOR';")

    primary_author = ("select a.bill_id from capublic.bill_tbl a" + 
                      " join capublic.bill_version_authors_tbl b" + 
                      " where a.latest_bill_version_id = b.bill_version_id and" + 
                      " b.name = %s and" + 
                      " contribution = 'LEAD_AUTHOR' and" + 
                      " primary_author_flg = 'Y';")
    sql = {"sponsor":sponsorship,
           "lead": lead_author,
           "primary": primary_author}[author_type]

    cursor.execute(sql, sponsor)
    bill_ids = cursor.fetchall()
    bill_count = len(bill_ids)
    votes = defaultdict(list)
    for bill_id in bill_ids:
        # get votes on bill
        cursor.execute("SELECT legislator_name, vote_code from capublic.bill_detail_vote_tbl WHERE bill_id = %s;", (bill_id,))

        for vote in cursor.fetchall():
            votes[vote[0]].append(1 if vote[1] == 'AYE' else -1)
    return votes, bill_count
        


In [ ]:
# Get Party affiliations
def get_legislators():
    sql = "select author_name, party from capublic.legislator_tbl;"
    cursor.execute(sql)
    data = list(cursor.fetchall())
    return pd.DataFrame(data, columns=["Name", "Party"])


In [ ]:
# caclculate score of voting for legislator's bills
def calc_score_data(votes, legislators):
    scores = []
    max_vote_count = 0
    for l in votes:
        if len(votes[l]):
            score = sum(votes[l])/len(votes[l])
            vote_count = len(votes[l])
            max_vote_count = max(max_vote_count, vote_count)
            scores.append([l, score, vote_count,])
    scores = pd.DataFrame(scores, columns=['Name','Score','Vote Count'])
    data = pd.merge(legislators, scores)
    data['% Bills Voted'] = data['Vote Count'].apply(lambda x: (x/max_vote_count)*100)
    # Remove outlier
    data = data[[(x > 10) for x in data['% Bills Voted']]]
    return data



In [ ]:
def sponsorship_report(sponsor, legislators):
    # sponsor
    votes, bill_count = get_votes(sponsor, author_type='sponsor')
    data = calc_score_data(votes, legislators)
    title= f"{bill_count} bills with {sponsor} as sponsor"
    return data, title
    
def lead_report(sponsor, legislators):
    # lead
    votes, bill_count = get_votes(sponsor, author_type='lead')
    data = calc_score_data(votes, legislators)
    title= f"{bill_count} bills with {sponsor} as lead author"
    return data, title
    
def primary_report(sponsor, legislators):
    # primary
    votes, bill_count = get_votes(sponsor, author_type='primary')
    data = calc_score_data(votes, legislators)
    title= f"{bill_count} bills with {sponsor} as primary lead author"
    return data, title

In [ ]:
def report_on_sponsor(sponsor):
    sponsor_row = legislators.loc[legislators['Name'] == sponsor]
   # print(sponsor_row)
    sponsor_party = sponsor_row['Party'].values[0]


    fig, axarr = plt.subplots(2, 2)
    fig.suptitle(f"Votes on bills with {sponsor} ({sponsor_party}) as sponsor/author", fontsize=10, fontweight='bold')


    data, title = sponsorship_report(sponsor, legislators)
    do_plot_pyplot(data, sponsor, title, axarr, 0, 0)

    axarr[0,1].axis('off')

    data, title = lead_report(sponsor, legislators)
    do_plot_pyplot(data, sponsor, title, axarr, 1, 0)

    data, title = primary_report(sponsor, legislators)
    do_plot_pyplot(data, sponsor, title, axarr, 1, 1)
    
    fig.tight_layout()
    fig.subplots_adjust(top=0.88)


In [ ]:
def do_plot_pyplot(data, sponsor, title, axarr, i, j):
    
    colors = ["#4878CF" if x == "DEM" else "#D65F5F" for x in data["Party"] ]
    #area = data["Vote Count"]/(max(data["Vote Count"])-min(data["Vote Count"])) * 
    ax = axarr[i,j]

    ax.set_title(title, fontsize=6)
    
    ax.scatter('Score', "% Bills Voted",data=data, s=0.3, c=colors, alpha=1)
    ax.set_xlabel("Score ( [AYEs - NOEs]/[total votes taken])", fontsize=5)
    ax.tick_params('both', labelsize=5)
    sponsor_row = data[data['Name'] == sponsor]

    try:
        ax.annotate(
            sponsor_row['Name'].values[0],
            xy= (sponsor_row['Score'].values[0], 
                 sponsor_row['% Bills Voted'].values[0]),
            xytext=(24,8),
            textcoords='offset points', ha='right', va='center',
            bbox=dict(boxstyle='round,pad=0.3', fc='yellow', alpha=0.3),
            arrowprops=dict(arrowstyle = '->', connectionstyle='arc3,rad=0'),
            fontsize=4
       )
    except Exception as e:
        print(e)
        print(data[data['Name'] == sponsor])
        
    if SHOW_ALL_LABELS:
        for _,row in data.iterrows():    
            ax.annotate(
                row['Name'],
                xy= (row['Score'], 
                     row['% Bills Voted']),
                xytext=(0,0),
                textcoords='offset points', ha='left', va='bottom',
                #bbox=dict(boxstyle='round,pad=0.3', fc='yellow', alpha=0.3),
                #arrowprops=dict(arrowstyle = '->', connectionstyle='arc3,rad=0'),
                fontsize=2
            )




In [ ]:
# All legislators
SHOW_ALL_LABELS = False

file_path = os.path.join(local_root, "output/2017-votes-report.pdf")
file_writer = PdfPages(file_path)

legislators = get_legislators()
for sponsor in legislators['Name']:
    sys.stdout.write(f"\rProcessing {sponsor}")
    sys.stdout.flush()
    fig = report_on_sponsor(sponsor)
    file_writer.savefig(bbox_inches='tight')

file_writer.close()
plt.close('all')
print("\nDone")

In [ ]:
# Do single legislator for faster testing: Chiu 
SHOW_ALL_LABELS = False

legislators = get_legislators()

file_path = os.path.join(local_root, "/output/test_chiu.pdf")
file_writer = PdfPages(file_path)
sponsor = 'Chiu'
report_on_sponsor(sponsor)



file_writer.savefig(bbox_inches='tight')
    
file_writer.close()
plt.show()
plt.close('all')
print("Done")